In [4]:
#ingestion.py

import os
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader, WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
#Text Splitters are classes for splitting text.

load_dotenv()

def charger_donnees():
    print("--- Début du chargement ---")
    
    # A. Chargement du PDF (Cours de Maths)
    # Assurez-vous que le fichier est dans le même dossier
    pdf_path = "Poly-Thermo-v1.pdf" 
    if os.path.exists(pdf_path):
        loader_pdf = PyPDFLoader(pdf_path)
        docs_pdf = loader_pdf.load()
        print(f"PDF chargé : {len(docs_pdf)} pages")
    else:
        print("Fichier PDF non trouvé, saut de cette étape.")
        docs_pdf = []

    # B. Chargement de Wikipédia (Espace de Banach)
    url = "https://fr.wikipedia.org/wiki/Espace_de_Banach"
    loader_web = WebBaseLoader(url)
    docs_web = loader_web.load()
    print(f"URL chargée : {url}")

    # Fusion des documents
    tous_les_docs = docs_pdf + docs_web
    

    # C. Découpage (Chunking)
    # On utilise des chunks de 1000 caractères avec un recouvrement 
    # pour ne pas couper les phrases au milieu.
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=150,
        add_start_index=True # Important pour citer la source précise
    )
    
    fragments = text_splitter.split_documents(tous_les_docs)
    print(f"Nombre total de fragments créés : {len(fragments)}")
    
    return fragments

if __name__ == "__main__":
    fragments = charger_donnees()
    # Test : affiche le contenu du premier fragment
    if fragments:
        print("\nExemple de fragment :")
        print(fragments[0].page_content[:200] + "...")

''' 
Le résumé de cette partie de script:

Le bouton "ON/OFF" automatique :

ON : Quand tu lances python ingestion.py, l'assertion est vraie. Le script s'exécute, 
charge le PDF et Wikipédia, et affiche tes tests.

OFF : Quand plus tard tu utiliseras ce fichier comme une "bibliothèque" 
(en faisant import ingestion dans un autre script), l'assertion sera fausse. 
Le script ne se lancera pas tout seul.
'''

USER_AGENT environment variable not set, consider setting it to identify your requests.


--- Début du chargement ---
PDF chargé : 119 pages
URL chargée : https://fr.wikipedia.org/wiki/Espace_de_Banach
Nombre total de fragments créés : 252

Exemple de fragment :
THERMODYNAMIQUE
J. SEBILLEAU
1...


In [5]:
#vectorisation.py


#----- J'en suis ici pour lacomprehension du code ( APPROFONDIR AU MAX !!)
#----- A CL CL A CHAQUE FOIS LA OU JE M'ARRETE
#Puis donner les bonnes e,trées pour la veille ( sites, api etc... N8N eventuellement )
#Finir par un beau front avec des KPIS et des graphs
# Ne pas oublier de faire des test avec des kpi's pour la prez
# Globalement a la fin reprendre toutes les notes ( les 2 pdf typst et en faire un gros pour ne rien oublier de ce qui à été fait)


from langchain_mistralai import MistralAIEmbeddings
from langchain_pinecone import PineconeVectorStore

# Initialisation des Embeddings (Le traducteur texte -> chiffres)
embeddings = MistralAIEmbeddings(model="mistral-embed", mistral_api_key=os.getenv("MISTRAL_API_KEY"))

index_name = "kpmg-veille"

# Envoi vers Pinecone
print("Envoi des vecteurs vers Pinecone... (cela peut prendre 1-2 min)")
vectorstore = PineconeVectorStore.from_documents(
    fragments, 
    embeddings, 
    index_name=index_name
)
print("Indexation terminée !")

Envoi des vecteurs vers Pinecone... (cela peut prendre 1-2 min)
Indexation terminée !


In [6]:
#chat_kpmg.py

import os
from dotenv import load_dotenv
from langchain_mistralai import ChatMistralAI, MistralAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

load_dotenv()

# 1. Connexion à l'index existant sur Pinecone
embeddings = MistralAIEmbeddings(model="mistral-embed")
index_name = "kpmg-veille"
vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3}) # On récupère les 3 meilleurs extraits

# 2. Configuration du "Cerveau" (Mistral)
llm = ChatMistralAI(model="mistral-medium", temperature=0)

# 3. Création du Prompt (Template) adapté aux exigences KPMG
template = """Vous êtes l'Assistant Intelligent de Veille Stratégique pour KPMG. 
Utilisez les extraits suivants pour répondre à la question. 
Si vous ne connaissez pas la réponse, dites-le. 
Citez TOUJOURS la source (ex: Page X du PDF ou URL Wikipédia).

CONTEXTE:
{context}

QUESTION: {question}

RÉPONSE ANALYTIQUE:"""

prompt = ChatPromptTemplate.from_template(template)

# 4. Construction de la Chaîne LCEL
def format_docs(docs):
    return "\n\n".join([f"Source: {d.metadata.get('source')} - Contenu: {d.page_content}" for d in docs])

chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 5. Test en conditions réelles
if __name__ == "__main__":
    question = "qu'est ce que la Thermodynamique statistique ? "
    print("--- Recherche en cours ---")
    print(chain.invoke(question))

--- Recherche en cours ---
D'après les extraits fournis du document **Poly-Thermo-v1.pdf**, la **thermodynamique statistique** peut être définie comme suit :

### **Définition et essence**
La thermodynamique statistique est une approche théorique qui **axiomatise le comportement statistique des molécules** constituant un système macroscopique (comme un gaz). Contrairement à la **thermodynamique classique** (empirique, basée sur l'expérience), elle part des **propriétés microscopiques** (mouvements, interactions des particules) pour en déduire les **grandeurs macroscopiques** (température, pression, entropie, etc.).

### **Principe clé**
- **Traitement statistique** : Grâce au **très grand nombre de molécules** (ex. : gaz avec *N* atomes), on peut décrire le système via un **petit nombre de grandeurs pertinentes** (ex. : vitesse moyenne, énergie cinétique), dont les **fluctuations statistiques sont négligeables**.
- **Lien micro ↔ macro** : Elle explique les lois thermodynamiques (comme

In [1]:
#Lire la documentation gradio : https://www.gradio.app/docs/gradio/chatinterface
#Plein de trucs a apprendre pour ameliorer le GUI
# Avantage ca s'incorpore dans des systèmes KPMG types web app etc avec gradio 


import gradio as gr
import os
from dotenv import load_dotenv
from langchain_mistralai import ChatMistralAI, MistralAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

load_dotenv()

# --- INITIALISATION RAG ---
embeddings = MistralAIEmbeddings(model="mistral-embed")
vectorstore = PineconeVectorStore(index_name="kpmg-veille", embedding=embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
llm = ChatMistralAI(model="mistral-medium", temperature=0)

template = """Vous êtes l'Expert Veille KPMG. Répondez à la question selon le contexte.
Citez les sources (ex: Page PDF ou URL).
CONTEXTE: {context}
QUESTION: {question}"""

prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    return "\n\n".join([f"Source: {d.metadata.get('source')} - {d.page_content}" for d in docs])

chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt | llm | StrOutputParser()
)

# --- FONCTION DE CHAT ---
def chat_response(message, history):
    try:
        response = chain.invoke(message)
        return response
    except Exception as e:
        return f"Erreur technique : {str(e)}"

# --- INTERFACE GRADIO ---
demo = gr.ChatInterface(
    fn=chat_response,
    title="Veilleur stratégique KPMG  ",
    description="Assistant RAG pour l'analyse de marché et la veille stratégique.",
    examples=["Qu'est-ce qu'un espace de Banach ?", "Quelles sont les tendances du marché ?"]
    )

if __name__ == "__main__":
    demo.launch(share=True) # share=True crée un lien public de 72h pour ton jury !

/Users/hobby/Desktop/Python/ML_projects/langchain_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://3c9485f389e4871fff.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
